In [2]:
# Lets fit a quadratic function with random noise:

f = x -> x^2 + 2x + 1 + 0.1randn()

m = 200
x = range(0,1; length=m) # makes 1000 evenly spaced points between 0 and 1
𝐟 = f.(x)

# We want to find the polynomial p(x) = p_0 + p_1*x + p_2*x^2  so that
# p(x[k]) ≈ f(x[k])
# in the sense that norm([p(x[k]) - f(x[k]) for k = 1:m]) == norm(p.(x) -f.(x))
# Make the Vandermonde matrix

V = [x[k]^j for k=1:m, j=0:2]
# Then p.(x) == V*c for c = [p_0,p_1,p_2]
# Thus we want to minimise
# norm(V*c - 𝐟)
# i.e.

c = V \ 𝐟
p = x -> c[1] + c[2]x + c[3]x^2


scatter(x, 𝐟)
plot!(x, p.(x))

LoadError: UndefVarError: scatter not defined

In [3]:
Q,R̂ = qr(V)
R̂ \ (Q'𝐟)[1:3]

3-element Vector{Float64}:
 1.0444692056288087
 1.8345658163181848
 1.1332314586458379